In [2]:
import pytorch_lightning

# check version
pytorch_lightning.__version__

'1.7.2'

In [3]:
import torch

# check version
torch.__version__

'1.12.1+cpu'

In [4]:
# ignore warnings
import warnings

warnings.filterwarnings('ignore')

In [6]:
from sklearn.datasets import load_iris
from torch.utils.data import DataLoader, TensorDataset, random_split

# load iris dataset
x, t = load_iris(return_X_y=True)

# transform to the format for learning with PyTorch
x = torch.tensor(x, dtype=torch.float32)
t = torch.tensor(t, dtype=torch.int64)

# transform input value and target value together to a object "dataset"
dataset = TensorDataset(x, t)

# set sample num of each dataset
# train : val : test = 60% : 20% : 20%
n_train = int(len(dataset) * 0.6)
n_val = int(len(dataset) * 0.2)
n_test = len(dataset) - n_train - n_val

# because of random split, fix random number seed to maintain reproductivity
torch.manual_seed(0)

# split dataset
train, val, test = random_split(dataset, [n_train, n_val, n_test])

# definition of batch size
batch_size = 32

# preparing DataLoader
# shuffle is False in default, so only train dataset is set True
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)

In [7]:
import pytorch_lightning as pl
import torch.nn as nn
import torch.nn.functional as F

class Net(pl.LightningModule):
    def __init__(self):
        super().__init__()

        self.bn = nn.BatchNorm1d(4)
        self.fc1 = nn.Linear(4, 4)
        self.fc2 = nn.Linear(4, 3)

    def forward(self, x):
        h = self.bn(x)
        h = self.fc1(h)
        h = F.relu(h)
        h = self.fc2(h)
        return h

    def training_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=0.01)
        return optimizer

In [10]:
# fix random number seed including GPU
pl.seed_everything(0)
net = Net()

# definition of trainer
trainer = pl.Trainer(max_epochs=5)

# learning
trainer.fit(net, train_loader, val_loader)

Global seed set to 0
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type        | Params
-------------------------------------
0 | bn   | BatchNorm1d | 8     
1 | fc1  | Linear      | 20    
2 | fc2  | Linear      | 15    
-------------------------------------
43        Trainable params
0         Non-trainable params
43        Total params
0.000     Total estimated model params size (MB)


Epoch 4: 100%|██████████| 2/2 [00:00<00:00, 50.09it/s, loss=1.04, v_num=2] 

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 2/2 [00:00<00:00, 36.70it/s, loss=1.04, v_num=2]


In [11]:
import tensorboard

tensorboard.__version__

'2.10.0'

In [15]:
import torchmetrics

class Net(pl.LightningModule):
    def __init__(self):
        super().__init__()

        self.bn = nn.BatchNorm1d(4)
        self.fc1 = nn.Linear(4, 4)
        self.fc2 = nn.Linear(4, 3)

        self.train_acc = torchmetrics.Accuracy()

    def forward(self, x):
        h = self.bn(x)
        h = self.fc1(h)
        h = F.relu(h)
        h = self.fc2(h)
        return h

    def training_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
        # add log
        # if on_step==True, the value of each iteration is recorded(default: True)
        # if on_epoch==True, the value of each epoch is recorded(default: False)
        self.log('train_loss', loss, on_step=True, on_epoch=True)
        self.log('train_acc', self.train_acc(y, t), on_step=True, on_epoch=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=0.01)
        return optimizer

In [17]:
# learning
pl.seed_everything(0)
net = Net()
trainer = pl.Trainer(max_epochs=30)
trainer.fit(net, train_loader, val_loader)

Global seed set to 0
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name      | Type        | Params
------------------------------------------
0 | bn        | BatchNorm1d | 8     
1 | fc1       | Linear      | 20    
2 | fc2       | Linear      | 15    
3 | train_acc | Accuracy    | 0     
------------------------------------------
43        Trainable params
0         Non-trainable params
43        Total params
0.000     Total estimated model params size (MB)


Epoch 29: 100%|██████████| 2/2 [00:00<00:00, 19.50it/s, loss=0.937, v_num=0]

`Trainer.fit` stopped: `max_epochs=30` reached.


Epoch 29: 100%|██████████| 2/2 [00:00<00:00, 16.91it/s, loss=0.937, v_num=0]


In [22]:
import torchmetrics

class Net(pl.LightningModule):
    def __init__(self):
        super().__init__()

        self.bn = nn.BatchNorm1d(4)
        self.fc1 = nn.Linear(4, 4)
        self.fc2 = nn.Linear(4, 3)

        self.train_acc = torchmetrics.Accuracy()
        self.val_acc = torchmetrics.Accuracy()
        self.test_acc = torchmetrics.Accuracy()

    def forward(self, x):
        h = self.bn(x)
        h = self.fc1(h)
        h = F.relu(h)
        h = self.fc2(h)
        return h

    def training_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
        # add log
        # if on_step==True, the value of each iteration is recorded(default: True)
        # if on_epoch==True, the value of each epoch is recorded(default: False)
        self.log('train_loss', loss, on_step=False, on_epoch=True)
        self.log('train_acc', self.train_acc(y, t), on_step=False, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x ,t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
        self.log('val_loss', loss, on_step=False, on_epoch=True)
        self.log('val_acc', self.val_acc(y, t), on_step=False, on_epoch=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
        self.log('test_loss', loss, on_step=False, on_epoch=True)
        self.log('test_acc', self.test_acc(y, t), on_step=False, on_epoch=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=0.01)
        return optimizer

In [23]:
# learning
pl.seed_everything(0)
net = Net()
trainer = pl.Trainer(max_epochs=30)
trainer.fit(net, train_loader, val_loader)

Global seed set to 0
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name      | Type        | Params
------------------------------------------
0 | bn        | BatchNorm1d | 8     
1 | fc1       | Linear      | 20    
2 | fc2       | Linear      | 15    
3 | train_acc | Accuracy    | 0     
4 | val_acc   | Accuracy    | 0     
5 | test_acc  | Accuracy    | 0     
------------------------------------------
43        Trainable params
0         Non-trainable params
43        Total params
0.000     Total estimated model params size (MB)


Epoch 29: 100%|██████████| 3/3 [00:00<00:00, 23.36it/s, loss=0.934, v_num=2]

`Trainer.fit` stopped: `max_epochs=30` reached.


Epoch 29: 100%|██████████| 3/3 [00:00<00:00, 20.94it/s, loss=0.934, v_num=2]


In [27]:
results = trainer.test(dataloaders=test_loader)

Restoring states from the checkpoint path at c:\Users\nagas\OneDrive - keio.jp\programming\Python_practice\deeplearning_implementation_introduction\deeplearning_implementation_introduction\lightning_logs\version_2\checkpoints\epoch=29-step=60.ckpt
Loaded model weights from checkpoint at c:\Users\nagas\OneDrive - keio.jp\programming\Python_practice\deeplearning_implementation_introduction\deeplearning_implementation_introduction\lightning_logs\version_2\checkpoints\epoch=29-step=60.ckpt


Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 66.19it/s] 
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.6000000238418579
        test_loss            0.925277054309845
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
